In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# dataset1

In [ ]:
import pandas as pd
df1 = pd.read_csv('/content/drive/MyDrive/Antifungal code/D1 code + dataset/D1 train all.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Antifungal code/D1 code + dataset/D1 test all.csv')

In [ ]:
import re

def extract_feature_names(data):
    # Assuming data is a list of feature names
    feature_names = set()

    for feature in data:
        match = re.search(r'^[a-zA-Z_]+', feature)
        if match:
            feature_names.add(match.group(0))

    return list(feature_names)

# Example usage
dataset_features = ["feature1_123", "feature2_456", "feature3_789", "another_feature_987"]
unique_feature_names = extract_feature_names(df1)

print("Unique Feature Names:", unique_feature_names)

Unique Feature Names: ['AAC_K', 'AAC_I', 'ngrams', 'AAC_C', 'AAC_L', 'binary', 'AAC_N', 'QSO', 'PAAC', 'AAC_D', 'AAC_M', 'AAC_V', 'AAC_Y', 'moran', 'AAC_P', 'AAC_G', 'AAC_R', 'AAC_H', 'AAC_A', 'Target', 'entropy', 'ctdc', 'AAC_F', 'AAC_T', 'AAC_E', 'AAC_S', 'AAC_W', 'AAC_Q', 'aaindex']


# Shap dataset analysis

In [ ]:
import pandas as pd

# Assuming your dataset is loaded into a variable named 'df'
# Replace 'your_dataset.csv' with the actual path to your dataset file if it's not loaded yet
# df = pd.read_csv('your_dataset.csv')

# List of columns to drop
columns_to_drop = ['ctdc1', 'ctdc19', 'ctdc32', 'binary1485', 'ctdc30', 'ctdc23', 'AAC_C',
                   'ctdc4', 'QSO2_SC_C', 'ctdc5', 'QSO2_G_C', 'ctdc20', 'ctdc2', 'ngrams222',
                   'ngrams162', 'ctdc6', 'ngrams29', 'ngrams289', 'QSO2_SC_K', 'aaindex321']
df1 = df1.iloc[:, 1:]
df2 = df2.iloc[:, 1:]
# Drop the columns
train = df1.drop(columns=columns_to_drop)
test = df2.drop(columns=columns_to_drop)

X_train = train.drop('Target', axis=1)
y_train = train['Target']

X_test = test.drop('Target', axis=1)
y_test = test['Target']


## EXtra tree

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve



# Create CatBoost classifier
ex = ExtraTreesClassifier(n_estimators=100, random_state=10)


# Train the model
ex.fit(X_train, y_train)

# Make predictions on the test set
test_predictions = ex.predict(X_test)

# Get predicted probabilities for calculating AUC score
test_probabilities = ex.predict_proba(X_test)[:, 1]

# Evaluate the performance
test_accuracy = accuracy_score(y_test, test_predictions)
conf_matrix = confusion_matrix(y_test, test_predictions)
roc_auc = roc_auc_score(y_test, test_probabilities)

# Calculate sensitivity and specificity from confusion matrix
true_negative, false_positive, false_negative, true_positive = conf_matrix.ravel()
sensitivity = true_positive / (true_positive + false_negative)
specificity = true_negative / (true_negative + false_positive)

from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

# Print the results

print("\nConfusion Matrix:")
print(conf_matrix)

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")




Confusion Matrix:
[[278  13]
 [ 34 257]]
Testing Accuracy: 0.9192439862542955

AUC Score: 0.9807
Sensitivity: 0.8832
Specificity: 0.9553
MCC Score: 0.8407
Kappa Score: 0.8385


## catboost

In [ ]:
!pip install CatBoost

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Create CatBoost classifier
catboost_model = CatBoostClassifier(iterations=100, depth=10, learning_rate=0.1, loss_function='Logloss', random_seed=42)

# Train the model
catboost_model.fit(X_train, y_train)

# Make predictions on the test set
test_predictions = catboost_model.predict(X_test)

# Get predicted probabilities for calculating AUC score
test_probabilities = catboost_model.predict_proba(X_test)[:, 1]

# Evaluate the performance
test_accuracy = accuracy_score(y_test, test_predictions)
conf_matrix = confusion_matrix(y_test, test_predictions)
roc_auc = roc_auc_score(y_test, test_probabilities)

# Calculate sensitivity and specificity from confusion matrix
true_negative, false_positive, false_negative, true_positive = conf_matrix.ravel()
sensitivity = true_positive / (true_positive + false_negative)
specificity = true_negative / (true_negative + false_positive)

from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

# Print the results

print("\nConfusion Matrix:")
print(conf_matrix)

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")



0:	learn: 0.5988310	total: 3.76s	remaining: 6m 12s
1:	learn: 0.5044923	total: 5.74s	remaining: 4m 41s
2:	learn: 0.4270873	total: 7.77s	remaining: 4m 11s
3:	learn: 0.3667414	total: 10.9s	remaining: 4m 21s
4:	learn: 0.3262207	total: 14.7s	remaining: 4m 38s
5:	learn: 0.2945590	total: 19.7s	remaining: 5m 8s
6:	learn: 0.2640862	total: 23.6s	remaining: 5m 14s
7:	learn: 0.2392128	total: 27.3s	remaining: 5m 14s
8:	learn: 0.2211752	total: 32.9s	remaining: 5m 32s
9:	learn: 0.1986781	total: 36.3s	remaining: 5m 26s
10:	learn: 0.1841697	total: 39.5s	remaining: 5m 19s
11:	learn: 0.1697235	total: 43.9s	remaining: 5m 22s
12:	learn: 0.1579424	total: 47.7s	remaining: 5m 19s
13:	learn: 0.1478983	total: 50.4s	remaining: 5m 9s
14:	learn: 0.1382498	total: 53s	remaining: 5m
15:	learn: 0.1300300	total: 55.9s	remaining: 4m 53s
16:	learn: 0.1203007	total: 59.9s	remaining: 4m 52s
17:	learn: 0.1121986	total: 1m 2s	remaining: 4m 44s
18:	learn: 0.1066646	total: 1m 5s	remaining: 4m 37s
19:	learn: 0.1012653	total: 1m

## LGBM

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

# Create LightGBM classifier
lgbm = LGBMClassifier(num_leaves=150, max_depth=100, learning_rate=0.1, n_estimators=100, subsample=1.0, colsample_bytree=1.0, reg_alpha=30, reg_lambda=10)

# Train the model
lgbm.fit(X_train, y_train)

# Make predictions on the test set
test_predictions = lgbm.predict(X_test)

# Get predicted probabilities for calculating AUC score
test_probabilities = lgbm.predict_proba(X_test)[:, 1]

# Evaluate the performance
test_accuracy = accuracy_score(y_test, test_predictions)
conf_matrix = confusion_matrix(y_test, test_predictions)
roc_auc = roc_auc_score(y_test, test_probabilities)

# Calculate sensitivity and specificity from confusion matrix
true_negative, false_positive, false_negative, true_positive = conf_matrix.ravel()
sensitivity = true_positive / (true_positive + false_negative)
specificity = true_negative / (true_negative + false_positive)

from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

# Print the results

print("\nConfusion Matrix:")
print(conf_matrix)

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")



[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.122476 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 189976
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 2613
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

## MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

# Print the results

print("\nConfusion Matrix:")
print(conf_matrix)

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")


Epoch 1/400
59/59 [==============================] - 3s 20ms/step - loss: 1.1260 - accuracy: 0.5177 - val_loss: 0.6423 - val_accuracy: 0.6474
Epoch 2/400
59/59 [==============================] - 1s 17ms/step - loss: 0.8546 - accuracy: 0.5664 - val_loss: 0.5824 - val_accuracy: 0.8013
Epoch 3/400
59/59 [==============================] - 1s 17ms/step - loss: 0.6409 - accuracy: 0.6617 - val_loss: 0.4439 - val_accuracy: 0.8440
Epoch 4/400
59/59 [==============================] - 1s 16ms/step - loss: 0.4880 - accuracy: 0.7848 - val_loss: 0.3086 - val_accuracy: 0.8953
Epoch 5/400
59/59 [==============================] - 1s 17ms/step - loss: 0.3377 - accuracy: 0.8672 - val_loss: 0.2119 - val_accuracy: 0.9316
Epoch 6/400
59/59 [==============================] - 1s 17ms/step - loss: 0.2487 - accuracy: 0.9213 - val_loss: 0.1951 - val_accuracy: 0.9295
Epoch 7/400
59/59 [==============================] - 1s 22ms/step - loss: 0.1661 - accuracy: 0.9433 - val_loss: 0.1942 - val_accuracy: 0.9103
Epoch 

# separate dataset

In [ ]:
aac_test = df2[list(f)+ ['Target']]

In [ ]:
aac_train = df1[list(f)+ ['Target']]

In [ ]:
qso_train = df1[list(f)+ ['Target']]


In [ ]:
qso_test = df2[list(f)+ ['Target']]

In [ ]:
aa1_train = df1[list(f)+ ['Target']]

In [ ]:
aa1_test = df2[list(f)+ ['Target']]

In [ ]:
ctdc_train = df1[list(f)+ ['Target']]

In [ ]:
ctdc_test = df2[list(f)+ ['Target']]

In [ ]:
ent_train = df1[list(f)+ ['Target']]

In [ ]:
ent_test = df2[list(f)+ ['Target']]

In [ ]:
mr_train = df1[list(f)+ ['Target']]

In [ ]:
mr_test = df2[list(f)+ ['Target']]

In [ ]:
paac_train = df1[list(f)+ ['Target']]

In [ ]:
paac_test = df2[list(f)+ ['Target']]

In [ ]:
ng_train = df1[list(f)+ ['Target']]

In [ ]:
ng_test = df2[list(f)+ ['Target']]

In [ ]:
bin_train = df1[list(f)+ ['Target']]

In [ ]:
bin_test = df2[list(f)+ ['Target']]

# lazzyporedict

## aac individual

In [ ]:
X_train = aac_train.drop("Target", axis=1)
y_train = aac_train['Target']

X_test = aac_test.drop("Target", axis=1)
y_test = aac_test['Target']

### lazzy clasify

In [ ]:
!python -m pip install --upgrade pip


In [ ]:
pip install lazypredict

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [00:08<00:00,  5.78it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000862 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4727
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [00:08<00:00,  3.37it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.86               0.86     0.86      0.86   
LGBMClassifier                     0.85               0.85     0.85      0.85   
XGBClassifier                      0.84               0.84     0.84      0.84   
RandomForestClassifier             0.84               0.84     0.84      0.84   
LabelPropagation                   0.83               0.83     0.83      0.83   
LabelSpreading                     0.83               0.83     0.83      0.83   
NuSVC                              0.82               0.82     0.82      0.82   
SVC                                0.81               0.81     0.81      0.81   
KNeighborsClassifier               0.81               0.81     0.81      0.81   
BaggingClassifier                  0.80               0.80     0.80      0.80   
AdaBoostClassifier          

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 4s 15ms/step - loss: 0.6984 - accuracy: 0.5487 - val_loss: 0.6520 - val_accuracy: 0.6282
Epoch 2/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6459 - accuracy: 0.6274 - val_loss: 0.5784 - val_accuracy: 0.6987
Epoch 3/400
59/59 [==============================] - 0s 5ms/step - loss: 0.5824 - accuracy: 0.6970 - val_loss: 0.5497 - val_accuracy: 0.7179
Epoch 4/400
59/59 [==============================] - 0s 6ms/step - loss: 0.5694 - accuracy: 0.7243 - val_loss: 0.5173 - val_accuracy: 0.7265
Epoch 5/400
59/59 [==============================] - 0s 5ms/step - loss: 0.5336 - accuracy: 0.7473 - val_loss: 0.5084 - val_accuracy: 0.7372
Epoch 6/400
59/59 [==============================] - 0s 5ms/step - loss: 0.5260 - accuracy: 0.7703 - val_loss: 0.4827 - val_accuracy: 0.7543
Epoch 7/400
59/59 [==============================] - 0s 5ms/step - loss: 0.4793 - accuracy: 0.7827 - val_loss: 0.4891 - val_accuracy: 0.7607
Epoch 8/400


## QSO

In [ ]:
X_train = qso_train.drop("Target", axis=1)
y_train = qso_train['Target']

X_test = qso_test.drop("Target", axis=1)
y_test = qso_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [00:08<00:00,  2.03it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001073 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6271
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [00:09<00:00,  3.01it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.87               0.87     0.87      0.87   
XGBClassifier                      0.87               0.87     0.87      0.87   
LGBMClassifier                     0.85               0.85     0.85      0.85   
RandomForestClassifier             0.85               0.85     0.85      0.85   
SVC                                0.83               0.83     0.83      0.83   
LabelPropagation                   0.82               0.82     0.82      0.82   
LabelSpreading                     0.82               0.82     0.82      0.82   
NuSVC                              0.82               0.82     0.82      0.82   
BaggingClassifier                  0.82               0.82     0.82      0.82   
KNeighborsClassifier               0.81               0.81     0.81      0.81   
AdaBoostClassifier          

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 8ms/step - loss: 0.7094 - accuracy: 0.5460 - val_loss: 0.6346 - val_accuracy: 0.6859
Epoch 2/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6248 - accuracy: 0.6547 - val_loss: 0.5392 - val_accuracy: 0.6966
Epoch 3/400
59/59 [==============================] - 0s 5ms/step - loss: 0.5783 - accuracy: 0.7099 - val_loss: 0.4874 - val_accuracy: 0.7222
Epoch 4/400
59/59 [==============================] - 0s 6ms/step - loss: 0.5288 - accuracy: 0.7409 - val_loss: 0.4603 - val_accuracy: 0.7714
Epoch 5/400
59/59 [==============================] - 0s 6ms/step - loss: 0.4984 - accuracy: 0.7554 - val_loss: 0.4362 - val_accuracy: 0.7927
Epoch 6/400
59/59 [==============================] - 0s 5ms/step - loss: 0.4793 - accuracy: 0.7880 - val_loss: 0.4199 - val_accuracy: 0.8098
Epoch 7/400
59/59 [==============================] - 0s 6ms/step - loss: 0.4513 - accuracy: 0.8116 - val_loss: 0.4095 - val_accuracy: 0.8269
Epoch 8/400
5

## paac

In [ ]:
X_train = paac_train.drop("Target", axis=1)
y_train = paac_train['Target']

X_test = paac_test.drop("Target", axis=1)
y_test = paac_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [00:05<00:00,  5.52it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000271 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5237
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [00:05<00:00,  4.90it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
XGBClassifier                      0.86               0.86     0.86      0.86   
ExtraTreesClassifier               0.85               0.85     0.85      0.85   
LGBMClassifier                     0.85               0.85     0.85      0.85   
RandomForestClassifier             0.84               0.84     0.84      0.84   
LabelSpreading                     0.83               0.83     0.83      0.83   
LabelPropagation                   0.83               0.83     0.83      0.83   
KNeighborsClassifier               0.83               0.83     0.83      0.83   
SVC                                0.82               0.82     0.82      0.82   
NuSVC                              0.82               0.82     0.82      0.82   
BaggingClassifier                  0.81               0.81     0.81      0.81   
DecisionTreeClassifier      

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 9ms/step - loss: 0.7039 - accuracy: 0.5327 - val_loss: 0.6491 - val_accuracy: 0.6325
Epoch 2/400
59/59 [==============================] - 0s 6ms/step - loss: 0.6404 - accuracy: 0.6488 - val_loss: 0.5717 - val_accuracy: 0.7009
Epoch 3/400
59/59 [==============================] - 0s 7ms/step - loss: 0.6009 - accuracy: 0.6959 - val_loss: 0.5317 - val_accuracy: 0.7201
Epoch 4/400
59/59 [==============================] - 0s 6ms/step - loss: 0.5591 - accuracy: 0.7377 - val_loss: 0.5034 - val_accuracy: 0.7393
Epoch 5/400
59/59 [==============================] - 0s 6ms/step - loss: 0.5275 - accuracy: 0.7495 - val_loss: 0.4769 - val_accuracy: 0.7671
Epoch 6/400
59/59 [==============================] - 0s 7ms/step - loss: 0.5170 - accuracy: 0.7682 - val_loss: 0.4709 - val_accuracy: 0.7906
Epoch 7/400
59/59 [==============================] - 0s 6ms/step - loss: 0.4934 - accuracy: 0.7736 - val_loss: 0.4678 - val_accuracy: 0.7735
Epoch 8/400
5

## aaindex

In [ ]:
X_train = aa1_train.drop("Target", axis=1)
y_train = aa1_train['Target']

X_test = aa1_test.drop("Target", axis=1)
y_test = aa1_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [01:13<00:02,  2.75s/it]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019681 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 141011
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 553
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [01:22<00:00,  2.83s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
LGBMClassifier                     0.92               0.92     0.92      0.92   
XGBClassifier                      0.91               0.91     0.91      0.91   
RandomForestClassifier             0.89               0.89     0.89      0.89   
NuSVC                              0.88               0.88     0.88      0.88   
ExtraTreesClassifier               0.87               0.87     0.87      0.87   
SVC                                0.87               0.87     0.87      0.87   
BaggingClassifier                  0.85               0.85     0.85      0.85   
KNeighborsClassifier               0.85               0.85     0.85      0.85   
DecisionTreeClassifier             0.81               0.81     0.81      0.81   
AdaBoostClassifier                 0.80               0.80     0.80      0.80   
ExtraTreeClassifier         

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 13ms/step - loss: 0.9491 - accuracy: 0.5150 - val_loss: 0.6800 - val_accuracy: 0.5919
Epoch 2/400
59/59 [==============================] - 1s 9ms/step - loss: 0.8179 - accuracy: 0.5118 - val_loss: 0.6771 - val_accuracy: 0.6004
Epoch 3/400
59/59 [==============================] - 0s 7ms/step - loss: 0.7557 - accuracy: 0.5321 - val_loss: 0.6723 - val_accuracy: 0.6474
Epoch 4/400
59/59 [==============================] - 0s 8ms/step - loss: 0.7243 - accuracy: 0.5471 - val_loss: 0.6694 - val_accuracy: 0.6624
Epoch 5/400
59/59 [==============================] - 0s 8ms/step - loss: 0.7398 - accuracy: 0.5444 - val_loss: 0.6574 - val_accuracy: 0.6218
Epoch 6/400
59/59 [==============================] - 0s 8ms/step - loss: 0.6969 - accuracy: 0.5733 - val_loss: 0.6396 - val_accuracy: 0.6752
Epoch 7/400
59/59 [==============================] - 0s 7ms/step - loss: 0.6835 - accuracy: 0.6001 - val_loss: 0.6394 - val_accuracy: 0.7158
Epoch 8/400


## CTDC

In [ ]:
X_train = ctdc_train.drop("Target", axis=1)
y_train = ctdc_train['Target']

X_test = ctdc_test.drop("Target", axis=1)
y_test = ctdc_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [00:08<00:00,  4.20it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001436 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9926
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [00:09<00:00,  3.14it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.83               0.83     0.83      0.83   
LabelSpreading                     0.83               0.83     0.83      0.83   
LabelPropagation                   0.83               0.83     0.83      0.83   
LGBMClassifier                     0.82               0.82     0.82      0.82   
XGBClassifier                      0.82               0.82     0.82      0.82   
KNeighborsClassifier               0.81               0.81     0.81      0.81   
RandomForestClassifier             0.81               0.81     0.81      0.81   
NuSVC                              0.81               0.81     0.81      0.81   
SVC                                0.80               0.80     0.80      0.80   
BaggingClassifier                  0.78               0.78     0.78      0.78   
DecisionTreeClassifier      

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 9ms/step - loss: 0.6887 - accuracy: 0.5541 - val_loss: 0.5797 - val_accuracy: 0.7222
Epoch 2/400
59/59 [==============================] - 0s 6ms/step - loss: 0.6177 - accuracy: 0.6708 - val_loss: 0.5353 - val_accuracy: 0.7265
Epoch 3/400
59/59 [==============================] - 0s 6ms/step - loss: 0.5932 - accuracy: 0.6975 - val_loss: 0.5130 - val_accuracy: 0.7350
Epoch 4/400
59/59 [==============================] - 0s 6ms/step - loss: 0.5695 - accuracy: 0.7152 - val_loss: 0.5006 - val_accuracy: 0.7564
Epoch 5/400
59/59 [==============================] - 0s 5ms/step - loss: 0.5526 - accuracy: 0.7382 - val_loss: 0.5068 - val_accuracy: 0.7585
Epoch 6/400
59/59 [==============================] - 0s 5ms/step - loss: 0.5555 - accuracy: 0.7281 - val_loss: 0.4867 - val_accuracy: 0.7799
Epoch 7/400
59/59 [==============================] - 0s 5ms/step - loss: 0.5537 - accuracy: 0.7537 - val_loss: 0.4845 - val_accuracy: 0.7842
Epoch 8/400
5

## moran

In [ ]:
X_train = mr_train.drop("Target", axis=1)
y_train = mr_train['Target']

X_test = mr_test.drop("Target", axis=1)
y_test = mr_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

100%|██████████| 29/29 [00:04<00:00,  5.96it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000358 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
LabelSpreading                     0.67               0.67     0.67      0.67   
LabelPropagation                   0.66               0.66     0.66      0.66   
ExtraTreesClassifier               0.66               0.66     0.66      0.66   
RandomForestClassifier             0.65               0.65     0.65      0.65   
SVC                                0.63               0.63     0.63      0.63  

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 8ms/step - loss: 0.7024 - accuracy: 0.4995 - val_loss: 0.6823 - val_accuracy: 0.5534
Epoch 2/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6932 - accuracy: 0.5278 - val_loss: 0.6786 - val_accuracy: 0.5385
Epoch 3/400
59/59 [==============================] - 0s 6ms/step - loss: 0.6869 - accuracy: 0.5332 - val_loss: 0.6730 - val_accuracy: 0.5385
Epoch 4/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6826 - accuracy: 0.5476 - val_loss: 0.6722 - val_accuracy: 0.5577
Epoch 5/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6803 - accuracy: 0.5541 - val_loss: 0.6710 - val_accuracy: 0.5278
Epoch 6/400
59/59 [==============================] - 0s 6ms/step - loss: 0.6743 - accuracy: 0.5476 - val_loss: 0.6643 - val_accuracy: 0.5577
Epoch 7/400
59/59 [==============================] - 1s 9ms/step - loss: 0.6793 - accuracy: 0.5439 - val_loss: 0.6661 - val_accuracy: 0.5620
Epoch 8/400
5

## binary

In [ ]:
X_train = bin_train.drop("Target", axis=1)
y_train = bin_train['Target']

X_test = bin_test.drop("Target", axis=1)
y_test = bin_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [01:44<00:03,  3.25s/it]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027952 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4668
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 1556
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [01:46<00:00,  3.66s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.85               0.85     0.85      0.84   
SVC                                0.83               0.83     0.83      0.83   
NuSVC                              0.83               0.83     0.83      0.83   
RandomForestClassifier             0.82               0.82     0.82      0.82   
LGBMClassifier                     0.81               0.81     0.81      0.81   
XGBClassifier                      0.81               0.81     0.81      0.81   
SGDClassifier                      0.78               0.78     0.78      0.78   
Perceptron                         0.78               0.78     0.78      0.78   
BaggingClassifier                  0.77               0.77     0.77      0.76   
LogisticRegression                 0.77               0.77     0.77      0.77   
PassiveAggressiveClassifier 

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 20ms/step - loss: 0.9641 - accuracy: 0.5112 - val_loss: 0.6796 - val_accuracy: 0.5919
Epoch 2/400
59/59 [==============================] - 1s 14ms/step - loss: 0.7869 - accuracy: 0.5225 - val_loss: 0.6687 - val_accuracy: 0.6603
Epoch 3/400
59/59 [==============================] - 1s 17ms/step - loss: 0.7237 - accuracy: 0.5873 - val_loss: 0.6377 - val_accuracy: 0.7393
Epoch 4/400
59/59 [==============================] - 1s 20ms/step - loss: 0.6347 - accuracy: 0.6419 - val_loss: 0.5729 - val_accuracy: 0.7457
Epoch 5/400
59/59 [==============================] - 1s 22ms/step - loss: 0.5134 - accuracy: 0.7473 - val_loss: 0.4687 - val_accuracy: 0.7842
Epoch 6/400
59/59 [==============================] - 1s 25ms/step - loss: 0.3781 - accuracy: 0.8415 - val_loss: 0.4506 - val_accuracy: 0.7991
Epoch 7/400
59/59 [==============================] - 1s 17ms/step - loss: 0.2724 - accuracy: 0.8972 - val_loss: 0.4822 - val_accuracy: 0.7970
Epoch 

## ngrams

In [ ]:
X_train = ng_train.drop("Target", axis=1)
y_train = ng_train['Target']

X_test = ng_test.drop("Target", axis=1)
y_test = ng_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [00:21<00:00,  1.46it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011045 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 20966
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 399
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [00:22<00:00,  1.26it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
RandomForestClassifier             0.89               0.89     0.89      0.89   
ExtraTreesClassifier               0.88               0.88     0.88      0.88   
LGBMClassifier                     0.88               0.88     0.88      0.88   
XGBClassifier                      0.87               0.87     0.87      0.87   
NuSVC                              0.85               0.85     0.85      0.85   
SVC                                0.85               0.85     0.85      0.85   
QuadraticDiscriminantAnalysis      0.83               0.83     0.83      0.83   
BaggingClassifier                  0.83               0.83     0.83      0.83   
RidgeClassifierCV                  0.77               0.77     0.77      0.77   
LinearDiscriminantAnalysis         0.77               0.77     0.77      0.77   
RidgeClassifier             

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 12ms/step - loss: 0.8864 - accuracy: 0.5027 - val_loss: 0.6706 - val_accuracy: 0.6453
Epoch 2/400
59/59 [==============================] - 0s 8ms/step - loss: 0.7073 - accuracy: 0.5584 - val_loss: 0.6417 - val_accuracy: 0.6902
Epoch 3/400
59/59 [==============================] - 0s 8ms/step - loss: 0.6395 - accuracy: 0.6263 - val_loss: 0.5388 - val_accuracy: 0.7521
Epoch 4/400
59/59 [==============================] - 0s 8ms/step - loss: 0.5972 - accuracy: 0.6949 - val_loss: 0.4790 - val_accuracy: 0.7842
Epoch 5/400
59/59 [==============================] - 1s 10ms/step - loss: 0.4860 - accuracy: 0.7843 - val_loss: 0.4203 - val_accuracy: 0.7906
Epoch 6/400
59/59 [==============================] - 1s 14ms/step - loss: 0.4235 - accuracy: 0.8340 - val_loss: 0.4115 - val_accuracy: 0.8013
Epoch 7/400
59/59 [==============================] - 1s 13ms/step - loss: 0.3310 - accuracy: 0.8726 - val_loss: 0.3999 - val_accuracy: 0.8291
Epoch 8/4

## entropy

In [ ]:
X_train = ent_train.drop("Target", axis=1)
y_train = ent_train['Target']

X_test = ent_test.drop("Target", axis=1)
y_test = ent_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

100%|██████████| 29/29 [00:04<00:00,  7.23it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000070 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 1
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.60               0.60     0.60      0.60   
BaggingClassifier                  0.59               0.59     0.59      0.59   
DecisionTreeClassifier             0.59               0.59     0.59      0.59   
RandomForestClassifier             0.59               0.59     0.59      0.59   
ExtraTreeClassifier                0.58               0.58     0.58      0.58   

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 10ms/step - loss: 0.7002 - accuracy: 0.5086 - val_loss: 0.6862 - val_accuracy: 0.5150
Epoch 2/400
59/59 [==============================] - 1s 10ms/step - loss: 0.6908 - accuracy: 0.5096 - val_loss: 0.6805 - val_accuracy: 0.5598
Epoch 3/400
59/59 [==============================] - 1s 10ms/step - loss: 0.6928 - accuracy: 0.5219 - val_loss: 0.6822 - val_accuracy: 0.5962
Epoch 4/400
59/59 [==============================] - 1s 9ms/step - loss: 0.6929 - accuracy: 0.5187 - val_loss: 0.6858 - val_accuracy: 0.5705
Epoch 5/400
59/59 [==============================] - 1s 10ms/step - loss: 0.6873 - accuracy: 0.5487 - val_loss: 0.6785 - val_accuracy: 0.5662
Epoch 6/400
59/59 [==============================] - 1s 11ms/step - loss: 0.6871 - accuracy: 0.5418 - val_loss: 0.6774 - val_accuracy: 0.5983
Epoch 7/400
59/59 [==============================] - 1s 9ms/step - loss: 0.6854 - accuracy: 0.5530 - val_loss: 0.6791 - val_accuracy: 0.5833
Epoch 8/

# dataset2

In [ ]:
import pandas as pd
df1 = pd.read_csv('/content/drive/MyDrive/Antifungal code/D2 code+ dataset/d2_all_train.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Antifungal code/D2 code+ dataset/d2_all_test (1).csv')

## Shap dataset analysis

In [ ]:
import pandas as pd

# Assuming your dataset is loaded into a variable named 'df'
# Replace 'your_dataset.csv' with the actual path to your dataset file if it's not loaded yet
# df = pd.read_csv('your_dataset.csv')

# List of columns to drop
columns_to_drop = ['ctdc1', 'ctdc19', 'ctdc32', 'binary1485', 'ctdc30', 'ctdc23', 'AAC_C',
                   'ctdc4', 'QSO2_SC_C', 'ctdc5', 'QSO2_G_C', 'ctdc20', 'ctdc2', 'ngrams222',
                   'ngrams162', 'ctdc6', 'ngrams29', 'ngrams289', 'QSO2_SC_K', 'aaindex321']
df1 = df1.iloc[:, 1:]
df2 = df2.iloc[:, 1:]
# Drop the columns
train = df1.drop(columns=columns_to_drop)
test = df2.drop(columns=columns_to_drop)

# X_train = train.drop('Target', axis=1)
# y_train = train['Target']

# X_test = test.drop('Target', axis=1)
# y_test = test['Target']


### EXtra tree

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve



# Create CatBoost classifier
ex = ExtraTreesClassifier(n_estimators=100, random_state=10)


# Train the model
ex.fit(X_train, y_train)

# Make predictions on the test set
test_predictions = ex.predict(X_test)

# Get predicted probabilities for calculating AUC score
test_probabilities = ex.predict_proba(X_test)[:, 1]

# Evaluate the performance
test_accuracy = accuracy_score(y_test, test_predictions)
conf_matrix = confusion_matrix(y_test, test_predictions)
roc_auc = roc_auc_score(y_test, test_probabilities)

# Calculate sensitivity and specificity from confusion matrix
true_negative, false_positive, false_negative, true_positive = conf_matrix.ravel()
sensitivity = true_positive / (true_positive + false_negative)
specificity = true_negative / (true_negative + false_positive)

from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

# Print the results

print("\nConfusion Matrix:")
print(conf_matrix)

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")




Confusion Matrix:
[[261  24]
 [100 191]]
Testing Accuracy: 0.7847222222222222

AUC Score: 0.8814
Sensitivity: 0.6564
Specificity: 0.9158
MCC Score: 0.5914
Kappa Score: 0.5706


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but ExtraTreesClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but ExtraTreesClassifier was fitted without feature names
  warnings.warn(


### catboost

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Create CatBoost classifier
catboost_model = CatBoostClassifier(iterations=100, depth=10, learning_rate=0.1, loss_function='Logloss', random_seed=42)

# Train the model
catboost_model.fit(X_train, y_train)

# Make predictions on the test set
test_predictions = catboost_model.predict(X_test)

# Get predicted probabilities for calculating AUC score
test_probabilities = catboost_model.predict_proba(X_test)[:, 1]

# Evaluate the performance
test_accuracy = accuracy_score(y_test, test_predictions)
conf_matrix = confusion_matrix(y_test, test_predictions)
roc_auc = roc_auc_score(y_test, test_probabilities)

# Calculate sensitivity and specificity from confusion matrix
true_negative, false_positive, false_negative, true_positive = conf_matrix.ravel()
sensitivity = true_positive / (true_positive + false_negative)
specificity = true_negative / (true_negative + false_positive)

from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

# Print the results

print("\nConfusion Matrix:")
print(conf_matrix)

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")



0:	learn: 0.5934849	total: 2.31s	remaining: 3m 49s
1:	learn: 0.5075877	total: 5.23s	remaining: 4m 16s
2:	learn: 0.4474645	total: 7.06s	remaining: 3m 48s
3:	learn: 0.3992780	total: 8.95s	remaining: 3m 34s
4:	learn: 0.3630500	total: 10.8s	remaining: 3m 25s
5:	learn: 0.3244169	total: 12.6s	remaining: 3m 17s
6:	learn: 0.2908511	total: 14.5s	remaining: 3m 12s
7:	learn: 0.2686748	total: 17.2s	remaining: 3m 17s
8:	learn: 0.2442512	total: 19.9s	remaining: 3m 20s
9:	learn: 0.2219995	total: 21.7s	remaining: 3m 15s
10:	learn: 0.2023502	total: 23.6s	remaining: 3m 11s
11:	learn: 0.1871643	total: 25.5s	remaining: 3m 7s
12:	learn: 0.1724165	total: 27.4s	remaining: 3m 3s
13:	learn: 0.1595909	total: 29.4s	remaining: 3m
14:	learn: 0.1493440	total: 32.5s	remaining: 3m 4s
15:	learn: 0.1405880	total: 34.6s	remaining: 3m 1s
16:	learn: 0.1294150	total: 36.5s	remaining: 2m 58s
17:	learn: 0.1213560	total: 38.5s	remaining: 2m 55s
18:	learn: 0.1144771	total: 40.4s	remaining: 2m 52s
19:	learn: 0.1082221	total: 42

### LGBM

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

# Create LightGBM classifier
lgbm = LGBMClassifier(num_leaves=150, max_depth=100, learning_rate=0.1, n_estimators=100, subsample=1.0, colsample_bytree=1.0, reg_alpha=30, reg_lambda=10)

# Train the model
lgbm.fit(X_train, y_train)

# Make predictions on the test set
test_predictions = lgbm.predict(X_test)

# Get predicted probabilities for calculating AUC score
test_probabilities = lgbm.predict_proba(X_test)[:, 1]

# Evaluate the performance
test_accuracy = accuracy_score(y_test, test_predictions)
conf_matrix = confusion_matrix(y_test, test_predictions)
roc_auc = roc_auc_score(y_test, test_probabilities)

# Calculate sensitivity and specificity from confusion matrix
true_negative, false_positive, false_negative, true_positive = conf_matrix.ravel()
sensitivity = true_positive / (true_positive + false_negative)
specificity = true_negative / (true_negative + false_positive)

from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

# Print the results

print("\nConfusion Matrix:")
print(conf_matrix)

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")



[LightGBM] [Info] Number of positive: 937, number of negative: 931
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.062724 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 188820
[LightGBM] [Info] Number of data points in the train set: 1868, number of used features: 2539
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501606 -> initscore=0.006424
[LightGBM] [Info] Start training from score 0.006424
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

# Print the results

print("\nConfusion Matrix:")
print(conf_matrix)

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Epoch 1/400
47/47 [==============================] - 3s 31ms/step - loss: 0.9868 - accuracy: 0.5402 - val_loss: 0.5979 - val_accuracy: 0.8021
Epoch 2/400
47/47 [==============================] - 1s 17ms/step - loss: 0.7801 - accuracy: 0.6205 - val_loss: 0.4954 - val_accuracy: 0.8957
Epoch 3/400
47/47 [==============================] - 1s 17ms/step - loss: 0.5084 - accuracy: 0.7764 - val_loss: 0.2323 - val_accuracy: 0.9465
Epoch 4/400
47/47 [==============================] - 1s 17ms/step - loss: 0.2973 - accuracy: 0.8896 - val_loss: 0.0879 - val_accuracy: 0.9706
Epoch 5/400
47/47 [==============================] - 1s 18ms/step - loss: 0.1727 - accuracy: 0.9485 - val_loss: 0.0730 - val_accuracy: 0.9786
Epoch 6/400
47/47 [==============================] - 1s 18ms/step - loss: 0.1188 - accuracy: 0.9726 - val_loss: 0.0542 - val_accuracy: 0.9866
Epoch 7/400
47/47 [==============================] - 1s 18ms/step - loss: 0.1046 - accuracy: 0.9739 - val_loss: 0.0847 - val_accuracy: 0.9786
Epoch 

## column separation

In [ ]:
import pandas as pd

# Assuming your dataset is a DataFrame named 'df1' with columns as feature names
# Example feature names: 'binary1', 'binary2', ..., 'QSO2_G2', 'Target'

# Create a dictionary to store separated DataFrames
feature_groups = {}

# Iterate through columns (features) in the DataFrame
for column in df2.columns:
    # Extract the character name by splitting the feature name
    character_name = ''.join(filter(str.isalpha, column.split('_')[0]))

    # Check if the character name is already a key in the dictionary
    if character_name in feature_groups:
        # If yes, append the column to the existing DataFrame
        feature_groups[character_name][column] = df2[column]
    else:
        # If no, create a new DataFrame for that character name
        feature_groups[character_name] = df2[[column]]

# Now, feature_groups contains separated DataFrames based on character names
# Access each DataFrame using feature_groups['CHARACTER_NAME']

# Example usage:
# Access the DataFrame for features with character name 'binary'
f = feature_groups['ctdc']

### separate dataset

In [ ]:
aac_test = df2[list(f)+ ['Target']]

In [ ]:
aac_train = df1[list(f)+ ['Target']]

In [ ]:
qso_train = df1[list(f)+ ['Target']]


In [ ]:
qso_test = df2[list(f)+ ['Target']]

In [ ]:
aa1_train = df1[list(f)+ ['Target']]

In [ ]:
aa1_test = df2[list(f)+ ['Target']]

In [ ]:
ctdc_train = df1[list(f)+ ['Target']]

In [ ]:
ctdc_test = df2[list(f)+ ['Target']]

In [ ]:
ent_train = df1[list(f)+ ['Target']]

In [ ]:
ent_test = df2[list(f)+ ['Target']]

In [ ]:
mr_train = df1[list(f)+ ['Target']]

In [ ]:
mr_test = df2[list(f)+ ['Target']]

In [ ]:
paac_train = df1[list(f)+ ['Target']]

In [ ]:
paac_test = df2[list(f)+ ['Target']]

In [ ]:
ng_train = df1[list(f)+ ['Target']]

In [ ]:
ng_test = df2[list(f)+ ['Target']]

In [ ]:
bin_train = df1[list(f)+ ['Target']]

In [ ]:
bin_test = df2[list(f)+ ['Target']]

## lazzyporedict

## aac individual

In [ ]:
X_train = aac_train.drop("Target", axis=1)
y_train = aac_train['Target']

X_test = aac_test.drop("Target", axis=1)
y_test = aac_test['Target']

### lazzy clasify

In [ ]:
!python -m pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
pip install lazypredict

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 93%|█████████▎| 27/29 [00:04<00:00,  8.00it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000580 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4782
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [00:05<00:00,  5.36it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.92               0.92     0.92      0.92   
SVC                                0.91               0.91     0.91      0.91   
XGBClassifier                      0.91               0.91     0.91      0.91   
RandomForestClassifier             0.91               0.91     0.91      0.91   
LGBMClassifier                     0.90               0.90     0.90      0.90   
NuSVC                              0.90               0.90     0.90      0.90   
KNeighborsClassifier               0.89               0.89     0.89      0.89   
LinearSVC                          0.88               0.88     0.88      0.88   
RidgeClassifierCV                  0.88               0.88     0.88      0.88   
RidgeClassifier                    0.88               0.88     0.88      0.88   
LinearDiscriminantAnalysis  

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 8ms/step - loss: 0.6185 - accuracy: 0.6595 - val_loss: 0.3756 - val_accuracy: 0.8718
Epoch 2/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3694 - accuracy: 0.8613 - val_loss: 0.3442 - val_accuracy: 0.8825
Epoch 3/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3360 - accuracy: 0.8790 - val_loss: 0.3361 - val_accuracy: 0.8846
Epoch 4/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3185 - accuracy: 0.8945 - val_loss: 0.3426 - val_accuracy: 0.8803
Epoch 5/400
59/59 [==============================] - 0s 6ms/step - loss: 0.3189 - accuracy: 0.8935 - val_loss: 0.3313 - val_accuracy: 0.8846
Epoch 6/400
59/59 [==============================] - 0s 5ms/step - loss: 0.2868 - accuracy: 0.8967 - val_loss: 0.3542 - val_accuracy: 0.8782
Epoch 7/400
59/59 [==============================] - 0s 5ms/step - loss: 0.2803 - accuracy: 0.9095 - val_loss: 0.3497 - val_accuracy: 0.8846
Epoch 8/400
5

## QSO

In [ ]:
X_train = qso_train.drop("Target", axis=1)
y_train = qso_train['Target']

X_test = qso_test.drop("Target", axis=1)
y_test = qso_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 7ms/step - loss: 0.6087 - accuracy: 0.6692 - val_loss: 0.3401 - val_accuracy: 0.8846
Epoch 2/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3477 - accuracy: 0.8678 - val_loss: 0.2849 - val_accuracy: 0.9017
Epoch 3/400
59/59 [==============================] - 0s 6ms/step - loss: 0.2955 - accuracy: 0.8887 - val_loss: 0.2831 - val_accuracy: 0.9081
Epoch 4/400
59/59 [==============================] - 0s 5ms/step - loss: 0.2609 - accuracy: 0.9058 - val_loss: 0.2593 - val_accuracy: 0.9103
Epoch 5/400
59/59 [==============================] - 0s 5ms/step - loss: 0.2478 - accuracy: 0.8994 - val_loss: 0.2621 - val_accuracy: 0.9060
Epoch 6/400
59/59 [==============================] - 0s 5ms/step - loss: 0.2207 - accuracy: 0.9181 - val_loss: 0.2748 - val_accuracy: 0.9103
Epoch 7/400
59/59 [==============================] - 1s 9ms/step - loss: 0.2162 - accuracy: 0.9229 - val_loss: 0.2574 - val_accuracy: 0.9167
Epoch 8/400
5

## paac

In [ ]:
X_train = paac_train.drop("Target", axis=1)
y_train = paac_train['Target']

X_test = paac_test.drop("Target", axis=1)
y_test = paac_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 8ms/step - loss: 0.5971 - accuracy: 0.6799 - val_loss: 0.3876 - val_accuracy: 0.8483
Epoch 2/400
59/59 [==============================] - 0s 4ms/step - loss: 0.3887 - accuracy: 0.8522 - val_loss: 0.3502 - val_accuracy: 0.8697
Epoch 3/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3129 - accuracy: 0.8908 - val_loss: 0.3521 - val_accuracy: 0.8803
Epoch 4/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3128 - accuracy: 0.8870 - val_loss: 0.3286 - val_accuracy: 0.8868
Epoch 5/400
59/59 [==============================] - 0s 4ms/step - loss: 0.2917 - accuracy: 0.8961 - val_loss: 0.3311 - val_accuracy: 0.8889
Epoch 6/400
59/59 [==============================] - 0s 5ms/step - loss: 0.2957 - accuracy: 0.9026 - val_loss: 0.3422 - val_accuracy: 0.8825
Epoch 7/400
59/59 [==============================] - 0s 5ms/step - loss: 0.2557 - accuracy: 0.9090 - val_loss: 0.3603 - val_accuracy: 0.8846
Epoch 8/400
5

## aaindex

In [ ]:
X_train = aa1_train.drop("Target", axis=1)
y_train = aa1_train['Target']

X_test = aa1_test.drop("Target", axis=1)
y_test = aa1_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 11ms/step - loss: 0.9785 - accuracy: 0.5177 - val_loss: 0.6668 - val_accuracy: 0.6603
Epoch 2/400
59/59 [==============================] - 1s 10ms/step - loss: 0.7709 - accuracy: 0.5760 - val_loss: 0.6369 - val_accuracy: 0.7158
Epoch 3/400
59/59 [==============================] - 1s 12ms/step - loss: 0.7228 - accuracy: 0.6065 - val_loss: 0.5578 - val_accuracy: 0.7714
Epoch 4/400
59/59 [==============================] - 1s 11ms/step - loss: 0.6620 - accuracy: 0.6504 - val_loss: 0.5183 - val_accuracy: 0.7991
Epoch 5/400
59/59 [==============================] - 1s 12ms/step - loss: 0.5853 - accuracy: 0.7099 - val_loss: 0.4646 - val_accuracy: 0.8376
Epoch 6/400
59/59 [==============================] - 1s 12ms/step - loss: 0.5533 - accuracy: 0.7468 - val_loss: 0.4218 - val_accuracy: 0.8269
Epoch 7/400
59/59 [==============================] - 1s 9ms/step - loss: 0.5043 - accuracy: 0.7687 - val_loss: 0.3857 - val_accuracy: 0.8590
Epoch 8

## CTDC

In [ ]:
X_train = ctdc_train.drop("Target", axis=1)
y_train = ctdc_train['Target']

X_test = ctdc_test.drop("Target", axis=1)
y_test = ctdc_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 8ms/step - loss: 0.5831 - accuracy: 0.6970 - val_loss: 0.3161 - val_accuracy: 0.8889
Epoch 2/400
59/59 [==============================] - 0s 5ms/step - loss: 0.4143 - accuracy: 0.8539 - val_loss: 0.2791 - val_accuracy: 0.8953
Epoch 3/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3751 - accuracy: 0.8704 - val_loss: 0.2720 - val_accuracy: 0.8953
Epoch 4/400
59/59 [==============================] - 0s 4ms/step - loss: 0.3650 - accuracy: 0.8721 - val_loss: 0.2718 - val_accuracy: 0.9124
Epoch 5/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3445 - accuracy: 0.8892 - val_loss: 0.2909 - val_accuracy: 0.8932
Epoch 6/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3304 - accuracy: 0.8790 - val_loss: 0.2555 - val_accuracy: 0.9103
Epoch 7/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3207 - accuracy: 0.8881 - val_loss: 0.2588 - val_accuracy: 0.9124
Epoch 8/400
5

## moran

In [ ]:
X_train = mr_train.drop("Target", axis=1)
y_train = mr_train['Target']

X_test = mr_test.drop("Target", axis=1)
y_test = mr_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

100%|██████████| 29/29 [00:04<00:00,  6.09it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000206 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
LGBMClassifier                     0.61               0.61     0.61      0.61   
XGBClassifier                      0.60               0.60     0.60      0.60   
KNeighborsClassifier               0.60               0.60     0.60      0.60   
LabelPropagation                   0.60               0.60     0.60      0.60   
LabelSpreading                     0.60               0.60     0.60      0.60  

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 8ms/step - loss: 0.7020 - accuracy: 0.4957 - val_loss: 0.6806 - val_accuracy: 0.6004
Epoch 2/400
59/59 [==============================] - 0s 4ms/step - loss: 0.6884 - accuracy: 0.5541 - val_loss: 0.6739 - val_accuracy: 0.5919
Epoch 3/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6816 - accuracy: 0.5541 - val_loss: 0.6676 - val_accuracy: 0.5791
Epoch 4/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6841 - accuracy: 0.5391 - val_loss: 0.6682 - val_accuracy: 0.5726
Epoch 5/400
59/59 [==============================] - 0s 4ms/step - loss: 0.6807 - accuracy: 0.5396 - val_loss: 0.6633 - val_accuracy: 0.5962
Epoch 6/400
59/59 [==============================] - 0s 4ms/step - loss: 0.6658 - accuracy: 0.5766 - val_loss: 0.6639 - val_accuracy: 0.6004
Epoch 7/400
59/59 [==============================] - 0s 4ms/step - loss: 0.6693 - accuracy: 0.5889 - val_loss: 0.6617 - val_accuracy: 0.6154
Epoch 8/400
5

## binary

In [ ]:
X_train = bin_train.drop("Target", axis=1)
y_train = bin_train['Target']

X_test = bin_test.drop("Target", axis=1)
y_test = bin_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [01:19<00:02,  2.90s/it]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027383 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4650
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 1550
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [01:21<00:00,  2.80s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.92               0.92     0.92      0.92   
LGBMClassifier                     0.92               0.92     0.92      0.92   
XGBClassifier                      0.92               0.92     0.92      0.92   
RandomForestClassifier             0.91               0.91     0.91      0.91   
NuSVC                              0.90               0.91     0.91      0.90   
SVC                                0.90               0.90     0.90      0.90   
BernoulliNB                        0.90               0.90     0.90      0.90   
NearestCentroid                    0.89               0.89     0.89      0.89   
BaggingClassifier                  0.89               0.89     0.89      0.89   
Perceptron                         0.86               0.86     0.86      0.86   
SGDClassifier               

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 4s 38ms/step - loss: 0.9365 - accuracy: 0.4936 - val_loss: 0.6736 - val_accuracy: 0.6346
Epoch 2/400
59/59 [==============================] - 1s 25ms/step - loss: 0.7803 - accuracy: 0.5482 - val_loss: 0.6708 - val_accuracy: 0.6709
Epoch 3/400
59/59 [==============================] - 1s 24ms/step - loss: 0.6995 - accuracy: 0.5969 - val_loss: 0.5751 - val_accuracy: 0.8291
Epoch 4/400
59/59 [==============================] - 1s 22ms/step - loss: 0.5379 - accuracy: 0.7398 - val_loss: 0.3831 - val_accuracy: 0.8739
Epoch 5/400
59/59 [==============================] - 1s 17ms/step - loss: 0.3254 - accuracy: 0.8667 - val_loss: 0.2821 - val_accuracy: 0.9038
Epoch 6/400
59/59 [==============================] - 1s 13ms/step - loss: 0.2267 - accuracy: 0.9160 - val_loss: 0.2807 - val_accuracy: 0.8953
Epoch 7/400
59/59 [==============================] - 1s 14ms/step - loss: 0.1402 - accuracy: 0.9529 - val_loss: 0.3077 - val_accuracy: 0.9060
Epoch 

## ngrams

In [ ]:
X_train = ng_train.drop("Target", axis=1)
y_train = ng_train['Target']

X_test = ng_test.drop("Target", axis=1)
y_test = ng_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [00:16<00:00,  1.69it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013903 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 21889
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 397
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [00:17<00:00,  1.62it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
SVC                                0.93               0.93     0.93      0.93   
ExtraTreesClassifier               0.92               0.92     0.92      0.92   
NuSVC                              0.91               0.91     0.91      0.91   
XGBClassifier                      0.91               0.91     0.91      0.91   
RandomForestClassifier             0.91               0.91     0.91      0.91   
LGBMClassifier                     0.90               0.90     0.90      0.90   
RidgeClassifierCV                  0.90               0.90     0.90      0.90   
RidgeClassifier                    0.90               0.90     0.90      0.90   
LinearDiscriminantAnalysis         0.90               0.90     0.90      0.90   
LogisticRegression                 0.89               0.89     0.89      0.89   
QuadraticDiscriminantAnalysi

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 10ms/step - loss: 0.7832 - accuracy: 0.5551 - val_loss: 0.6219 - val_accuracy: 0.8098
Epoch 2/400
59/59 [==============================] - 0s 6ms/step - loss: 0.6360 - accuracy: 0.6681 - val_loss: 0.4000 - val_accuracy: 0.8739
Epoch 3/400
59/59 [==============================] - 0s 7ms/step - loss: 0.4189 - accuracy: 0.8249 - val_loss: 0.2851 - val_accuracy: 0.8868
Epoch 4/400
59/59 [==============================] - 0s 7ms/step - loss: 0.3021 - accuracy: 0.8881 - val_loss: 0.2750 - val_accuracy: 0.8953
Epoch 5/400
59/59 [==============================] - 0s 7ms/step - loss: 0.2404 - accuracy: 0.9160 - val_loss: 0.2758 - val_accuracy: 0.8953
Epoch 6/400
59/59 [==============================] - 0s 7ms/step - loss: 0.1782 - accuracy: 0.9427 - val_loss: 0.3105 - val_accuracy: 0.9017
Epoch 7/400
59/59 [==============================] - 0s 7ms/step - loss: 0.1526 - accuracy: 0.9491 - val_loss: 0.3743 - val_accuracy: 0.9038
Epoch 8/400


## entropy

In [ ]:
X_train = ent_train.drop("Target", axis=1)
y_train = ent_train['Target']

X_test = ent_test.drop("Target", axis=1)
y_test = ent_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

100%|██████████| 29/29 [00:03<00:00,  8.05it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000082 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 1
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
SVC                                0.55               0.54     0.54      0.53   
CalibratedClassifierCV             0.54               0.54     0.54      0.54   
KNeighborsClassifier               0.54               0.54     0.54      0.54   
BaggingClassifier                  0.54               0.54     0.54      0.54   
LabelSpreading                     0.54               0.53     0.53      0.51   

### proposed model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")


Epoch 1/400
59/59 [==============================] - 2s 9ms/step - loss: 0.6958 - accuracy: 0.5037 - val_loss: 0.6954 - val_accuracy: 0.5064
Epoch 2/400
59/59 [==============================] - 0s 4ms/step - loss: 0.6968 - accuracy: 0.4973 - val_loss: 0.6920 - val_accuracy: 0.5214
Epoch 3/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6986 - accuracy: 0.4984 - val_loss: 0.6931 - val_accuracy: 0.5192
Epoch 4/400
59/59 [==============================] - 0s 6ms/step - loss: 0.6957 - accuracy: 0.5016 - val_loss: 0.6932 - val_accuracy: 0.5043
Epoch 5/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6931 - accuracy: 0.5203 - val_loss: 0.6953 - val_accuracy: 0.5021
Epoch 6/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6974 - accuracy: 0.5086 - val_loss: 0.6927 - val_accuracy: 0.5085
Epoch 7/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6946 - accuracy: 0.5075 - val_loss: 0.6929 - val_accuracy: 0.5150
Epoch 8/400
5

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

F1 Score: 0.6683


# dataset3

In [ ]:
import pandas as pd
df1 = pd.read_csv('/content/drive/MyDrive/Antifungal code/D3 code+ dataset/d3_all_train.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Antifungal code/D3 code+ dataset/d3_all_test.csv')

## Shap dataset analysis

In [ ]:
import pandas as pd

# Assuming your dataset is loaded into a variable named 'df'
# Replace 'your_dataset.csv' with the actual path to your dataset file if it's not loaded yet
# df = pd.read_csv('your_dataset.csv')

# List of columns to drop
columns_to_drop = ['ctdc1', 'ctdc19', 'ctdc32', 'binary1485', 'ctdc30', 'ctdc23', 'AAC_C',
                   'ctdc4', 'QSO2_SC_C', 'ctdc5', 'QSO2_G_C', 'ctdc20', 'ctdc2', 'ngrams222',
                   'ngrams162', 'ctdc6', 'ngrams29', 'ngrams289', 'QSO2_SC_K', 'aaindex321']
df1 = df1.iloc[:, 1:]
df2 = df2.iloc[:, 1:]
# Drop the columns
train = df1.drop(columns=columns_to_drop)
test = df2.drop(columns=columns_to_drop)

X_train = train.drop('Target', axis=1)
y_train = train['Target']

X_test = test.drop('Target', axis=1)
y_test = test['Target']


### EXtra tree

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve



# Create CatBoost classifier
ex = ExtraTreesClassifier(n_estimators=100, random_state=10)


# Train the model
ex.fit(X_train, y_train)

# Make predictions on the test set
test_predictions = ex.predict(X_test)

# Get predicted probabilities for calculating AUC score
test_probabilities = ex.predict_proba(X_test)[:, 1]

# Evaluate the performance
test_accuracy = accuracy_score(y_test, test_predictions)
conf_matrix = confusion_matrix(y_test, test_predictions)
roc_auc = roc_auc_score(y_test, test_probabilities)

# Calculate sensitivity and specificity from confusion matrix
true_negative, false_positive, false_negative, true_positive = conf_matrix.ravel()
sensitivity = true_positive / (true_positive + false_negative)
specificity = true_negative / (true_negative + false_positive)

from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

# Print the results

print("\nConfusion Matrix:")
print(conf_matrix)

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")




Confusion Matrix:
[[279   8]
 [ 23 268]]
Testing Accuracy: 0.9463667820069204

AUC Score: 0.9880
Sensitivity: 0.9210
Specificity: 0.9721
MCC Score: 0.8940
Kappa Score: 0.8928


### catboost

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Create CatBoost classifier
catboost_model = CatBoostClassifier(iterations=100, depth=10, learning_rate=0.1, loss_function='Logloss', random_seed=42)

# Train the model
catboost_model.fit(X_train, y_train)

# Make predictions on the test set
test_predictions = catboost_model.predict(X_test)

# Get predicted probabilities for calculating AUC score
test_probabilities = catboost_model.predict_proba(X_test)[:, 1]

# Evaluate the performance
test_accuracy = accuracy_score(y_test, test_predictions)
conf_matrix = confusion_matrix(y_test, test_predictions)
roc_auc = roc_auc_score(y_test, test_probabilities)

# Calculate sensitivity and specificity from confusion matrix
true_negative, false_positive, false_negative, true_positive = conf_matrix.ravel()
sensitivity = true_positive / (true_positive + false_negative)
specificity = true_negative / (true_negative + false_positive)

from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

# Print the results

print("\nConfusion Matrix:")
print(conf_matrix)

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")



0:	learn: 0.6226728	total: 2.82s	remaining: 4m 38s
1:	learn: 0.5548308	total: 5.58s	remaining: 4m 33s
2:	learn: 0.5090712	total: 9.27s	remaining: 4m 59s
3:	learn: 0.4692347	total: 12.8s	remaining: 5m 6s
4:	learn: 0.4339484	total: 15.7s	remaining: 4m 59s
5:	learn: 0.3948321	total: 18.4s	remaining: 4m 47s
6:	learn: 0.3682408	total: 21.1s	remaining: 4m 40s
7:	learn: 0.3376031	total: 25.5s	remaining: 4m 53s
8:	learn: 0.3155513	total: 28.4s	remaining: 4m 47s
9:	learn: 0.2954315	total: 31s	remaining: 4m 39s
10:	learn: 0.2755574	total: 33.8s	remaining: 4m 33s
11:	learn: 0.2606486	total: 37.8s	remaining: 4m 37s
12:	learn: 0.2475172	total: 41.1s	remaining: 4m 34s
13:	learn: 0.2317227	total: 43.7s	remaining: 4m 28s
14:	learn: 0.2168942	total: 46.6s	remaining: 4m 24s
15:	learn: 0.2049054	total: 49.8s	remaining: 4m 21s
16:	learn: 0.1895159	total: 53.9s	remaining: 4m 23s
17:	learn: 0.1802771	total: 56.5s	remaining: 4m 17s
18:	learn: 0.1704257	total: 59.3s	remaining: 4m 12s
19:	learn: 0.1617693	tota

### LGBM

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

# Create LightGBM classifier
lgbm = LGBMClassifier(num_leaves=150, max_depth=100, learning_rate=0.1, n_estimators=100, subsample=1.0, colsample_bytree=1.0, reg_alpha=30, reg_lambda=10)

# Train the model
lgbm.fit(X_train, y_train)

# Make predictions on the test set
test_predictions = lgbm.predict(X_test)

# Get predicted probabilities for calculating AUC score
test_probabilities = lgbm.predict_proba(X_test)[:, 1]

# Evaluate the performance
test_accuracy = accuracy_score(y_test, test_predictions)
conf_matrix = confusion_matrix(y_test, test_predictions)
roc_auc = roc_auc_score(y_test, test_probabilities)

# Calculate sensitivity and specificity from confusion matrix
true_negative, false_positive, false_negative, true_positive = conf_matrix.ravel()
sensitivity = true_positive / (true_positive + false_negative)
specificity = true_negative / (true_negative + false_positive)

from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

# Print the results

print("\nConfusion Matrix:")
print(conf_matrix)

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")



[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.077039 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 190022
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 2629
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

# Print the results

print("\nConfusion Matrix:")
print(conf_matrix)

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")


Epoch 1/400
59/59 [==============================] - 2s 20ms/step - loss: 1.0686 - accuracy: 0.5193 - val_loss: 0.6449 - val_accuracy: 0.7073
Epoch 2/400
59/59 [==============================] - 1s 17ms/step - loss: 0.7942 - accuracy: 0.5782 - val_loss: 0.5932 - val_accuracy: 0.8013
Epoch 3/400
59/59 [==============================] - 1s 18ms/step - loss: 0.6662 - accuracy: 0.6713 - val_loss: 0.4273 - val_accuracy: 0.8996
Epoch 4/400
59/59 [==============================] - 1s 18ms/step - loss: 0.4910 - accuracy: 0.7757 - val_loss: 0.2591 - val_accuracy: 0.9359
Epoch 5/400
59/59 [==============================] - 1s 18ms/step - loss: 0.2983 - accuracy: 0.8897 - val_loss: 0.1662 - val_accuracy: 0.9380
Epoch 6/400
59/59 [==============================] - 2s 26ms/step - loss: 0.2046 - accuracy: 0.9422 - val_loss: 0.0944 - val_accuracy: 0.9679
Epoch 7/400
59/59 [==============================] - 2s 27ms/step - loss: 0.1560 - accuracy: 0.9486 - val_loss: 0.0784 - val_accuracy: 0.9679
Epoch 

## column separation

In [ ]:
import pandas as pd

# Assuming your dataset is a DataFrame named 'df1' with columns as feature names
# Example feature names: 'binary1', 'binary2', ..., 'QSO2_G2', 'Target'

# Create a dictionary to store separated DataFrames
feature_groups = {}

# Iterate through columns (features) in the DataFrame
for column in df2.columns:
    # Extract the character name by splitting the feature name
    character_name = ''.join(filter(str.isalpha, column.split('_')[0]))

    # Check if the character name is already a key in the dictionary
    if character_name in feature_groups:
        # If yes, append the column to the existing DataFrame
        feature_groups[character_name][column] = df2[column]
    else:
        # If no, create a new DataFrame for that character name
        feature_groups[character_name] = df2[[column]]

# Now, feature_groups contains separated DataFrames based on character names
# Access each DataFrame using feature_groups['CHARACTER_NAME']

# Example usage:
# Access the DataFrame for features with character name 'binary'
f = feature_groups['ctdc']

### separate dataset

In [ ]:
aac_test = df2[list(f)+ ['Target']]

In [ ]:
aac_train = df1[list(f)+ ['Target']]

In [ ]:
qso_train = df1[list(f)+ ['Target']]


In [ ]:
qso_test = df2[list(f)+ ['Target']]

In [ ]:
aa1_train = df1[list(f)+ ['Target']]

In [ ]:
aa1_test = df2[list(f)+ ['Target']]

In [ ]:
ctdc_train = df1[list(f)+ ['Target']]

In [ ]:
ctdc_test = df2[list(f)+ ['Target']]

In [ ]:
ent_train = df1[list(f)+ ['Target']]

In [ ]:
ent_test = df2[list(f)+ ['Target']]

In [ ]:
mr_train = df1[list(f)+ ['Target']]

In [ ]:
mr_test = df2[list(f)+ ['Target']]

In [ ]:
paac_train = df1[list(f)+ ['Target']]

In [ ]:
paac_test = df2[list(f)+ ['Target']]

In [ ]:
ng_train = df1[list(f)+ ['Target']]

In [ ]:
ng_test = df2[list(f)+ ['Target']]

In [ ]:
bin_train = df1[list(f)+ ['Target']]

In [ ]:
bin_test = df2[list(f)+ ['Target']]

## lazzyporedict

## aac individual

In [ ]:
X_train = aac_train.drop("Target", axis=1)
y_train = aac_train['Target']

X_test = aac_test.drop("Target", axis=1)
y_test = aac_test['Target']

### lazzy clasify

In [ ]:
!python -m pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
pip install lazypredict

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 93%|█████████▎| 27/29 [00:04<00:00,  8.00it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000580 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4782
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [00:05<00:00,  5.36it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.92               0.92     0.92      0.92   
SVC                                0.91               0.91     0.91      0.91   
XGBClassifier                      0.91               0.91     0.91      0.91   
RandomForestClassifier             0.91               0.91     0.91      0.91   
LGBMClassifier                     0.90               0.90     0.90      0.90   
NuSVC                              0.90               0.90     0.90      0.90   
KNeighborsClassifier               0.89               0.89     0.89      0.89   
LinearSVC                          0.88               0.88     0.88      0.88   
RidgeClassifierCV                  0.88               0.88     0.88      0.88   
RidgeClassifier                    0.88               0.88     0.88      0.88   
LinearDiscriminantAnalysis  

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 8ms/step - loss: 0.6185 - accuracy: 0.6595 - val_loss: 0.3756 - val_accuracy: 0.8718
Epoch 2/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3694 - accuracy: 0.8613 - val_loss: 0.3442 - val_accuracy: 0.8825
Epoch 3/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3360 - accuracy: 0.8790 - val_loss: 0.3361 - val_accuracy: 0.8846
Epoch 4/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3185 - accuracy: 0.8945 - val_loss: 0.3426 - val_accuracy: 0.8803
Epoch 5/400
59/59 [==============================] - 0s 6ms/step - loss: 0.3189 - accuracy: 0.8935 - val_loss: 0.3313 - val_accuracy: 0.8846
Epoch 6/400
59/59 [==============================] - 0s 5ms/step - loss: 0.2868 - accuracy: 0.8967 - val_loss: 0.3542 - val_accuracy: 0.8782
Epoch 7/400
59/59 [==============================] - 0s 5ms/step - loss: 0.2803 - accuracy: 0.9095 - val_loss: 0.3497 - val_accuracy: 0.8846
Epoch 8/400
5

## QSO

In [ ]:
X_train = qso_train.drop("Target", axis=1)
y_train = qso_train['Target']

X_test = qso_test.drop("Target", axis=1)
y_test = qso_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 7ms/step - loss: 0.6087 - accuracy: 0.6692 - val_loss: 0.3401 - val_accuracy: 0.8846
Epoch 2/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3477 - accuracy: 0.8678 - val_loss: 0.2849 - val_accuracy: 0.9017
Epoch 3/400
59/59 [==============================] - 0s 6ms/step - loss: 0.2955 - accuracy: 0.8887 - val_loss: 0.2831 - val_accuracy: 0.9081
Epoch 4/400
59/59 [==============================] - 0s 5ms/step - loss: 0.2609 - accuracy: 0.9058 - val_loss: 0.2593 - val_accuracy: 0.9103
Epoch 5/400
59/59 [==============================] - 0s 5ms/step - loss: 0.2478 - accuracy: 0.8994 - val_loss: 0.2621 - val_accuracy: 0.9060
Epoch 6/400
59/59 [==============================] - 0s 5ms/step - loss: 0.2207 - accuracy: 0.9181 - val_loss: 0.2748 - val_accuracy: 0.9103
Epoch 7/400
59/59 [==============================] - 1s 9ms/step - loss: 0.2162 - accuracy: 0.9229 - val_loss: 0.2574 - val_accuracy: 0.9167
Epoch 8/400
5

## paac

In [ ]:
X_train = paac_train.drop("Target", axis=1)
y_train = paac_train['Target']

X_test = paac_test.drop("Target", axis=1)
y_test = paac_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 8ms/step - loss: 0.5971 - accuracy: 0.6799 - val_loss: 0.3876 - val_accuracy: 0.8483
Epoch 2/400
59/59 [==============================] - 0s 4ms/step - loss: 0.3887 - accuracy: 0.8522 - val_loss: 0.3502 - val_accuracy: 0.8697
Epoch 3/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3129 - accuracy: 0.8908 - val_loss: 0.3521 - val_accuracy: 0.8803
Epoch 4/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3128 - accuracy: 0.8870 - val_loss: 0.3286 - val_accuracy: 0.8868
Epoch 5/400
59/59 [==============================] - 0s 4ms/step - loss: 0.2917 - accuracy: 0.8961 - val_loss: 0.3311 - val_accuracy: 0.8889
Epoch 6/400
59/59 [==============================] - 0s 5ms/step - loss: 0.2957 - accuracy: 0.9026 - val_loss: 0.3422 - val_accuracy: 0.8825
Epoch 7/400
59/59 [==============================] - 0s 5ms/step - loss: 0.2557 - accuracy: 0.9090 - val_loss: 0.3603 - val_accuracy: 0.8846
Epoch 8/400
5

## aaindex

In [ ]:
X_train = aa1_train.drop("Target", axis=1)
y_train = aa1_train['Target']

X_test = aa1_test.drop("Target", axis=1)
y_test = aa1_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 11ms/step - loss: 0.9785 - accuracy: 0.5177 - val_loss: 0.6668 - val_accuracy: 0.6603
Epoch 2/400
59/59 [==============================] - 1s 10ms/step - loss: 0.7709 - accuracy: 0.5760 - val_loss: 0.6369 - val_accuracy: 0.7158
Epoch 3/400
59/59 [==============================] - 1s 12ms/step - loss: 0.7228 - accuracy: 0.6065 - val_loss: 0.5578 - val_accuracy: 0.7714
Epoch 4/400
59/59 [==============================] - 1s 11ms/step - loss: 0.6620 - accuracy: 0.6504 - val_loss: 0.5183 - val_accuracy: 0.7991
Epoch 5/400
59/59 [==============================] - 1s 12ms/step - loss: 0.5853 - accuracy: 0.7099 - val_loss: 0.4646 - val_accuracy: 0.8376
Epoch 6/400
59/59 [==============================] - 1s 12ms/step - loss: 0.5533 - accuracy: 0.7468 - val_loss: 0.4218 - val_accuracy: 0.8269
Epoch 7/400
59/59 [==============================] - 1s 9ms/step - loss: 0.5043 - accuracy: 0.7687 - val_loss: 0.3857 - val_accuracy: 0.8590
Epoch 8

## CTDC

In [ ]:
X_train = ctdc_train.drop("Target", axis=1)
y_train = ctdc_train['Target']

X_test = ctdc_test.drop("Target", axis=1)
y_test = ctdc_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 8ms/step - loss: 0.5831 - accuracy: 0.6970 - val_loss: 0.3161 - val_accuracy: 0.8889
Epoch 2/400
59/59 [==============================] - 0s 5ms/step - loss: 0.4143 - accuracy: 0.8539 - val_loss: 0.2791 - val_accuracy: 0.8953
Epoch 3/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3751 - accuracy: 0.8704 - val_loss: 0.2720 - val_accuracy: 0.8953
Epoch 4/400
59/59 [==============================] - 0s 4ms/step - loss: 0.3650 - accuracy: 0.8721 - val_loss: 0.2718 - val_accuracy: 0.9124
Epoch 5/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3445 - accuracy: 0.8892 - val_loss: 0.2909 - val_accuracy: 0.8932
Epoch 6/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3304 - accuracy: 0.8790 - val_loss: 0.2555 - val_accuracy: 0.9103
Epoch 7/400
59/59 [==============================] - 0s 5ms/step - loss: 0.3207 - accuracy: 0.8881 - val_loss: 0.2588 - val_accuracy: 0.9124
Epoch 8/400
5

## moran

In [ ]:
X_train = mr_train.drop("Target", axis=1)
y_train = mr_train['Target']

X_test = mr_test.drop("Target", axis=1)
y_test = mr_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

100%|██████████| 29/29 [00:04<00:00,  6.09it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000206 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
LGBMClassifier                     0.61               0.61     0.61      0.61   
XGBClassifier                      0.60               0.60     0.60      0.60   
KNeighborsClassifier               0.60               0.60     0.60      0.60   
LabelPropagation                   0.60               0.60     0.60      0.60   
LabelSpreading                     0.60               0.60     0.60      0.60  

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 8ms/step - loss: 0.7020 - accuracy: 0.4957 - val_loss: 0.6806 - val_accuracy: 0.6004
Epoch 2/400
59/59 [==============================] - 0s 4ms/step - loss: 0.6884 - accuracy: 0.5541 - val_loss: 0.6739 - val_accuracy: 0.5919
Epoch 3/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6816 - accuracy: 0.5541 - val_loss: 0.6676 - val_accuracy: 0.5791
Epoch 4/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6841 - accuracy: 0.5391 - val_loss: 0.6682 - val_accuracy: 0.5726
Epoch 5/400
59/59 [==============================] - 0s 4ms/step - loss: 0.6807 - accuracy: 0.5396 - val_loss: 0.6633 - val_accuracy: 0.5962
Epoch 6/400
59/59 [==============================] - 0s 4ms/step - loss: 0.6658 - accuracy: 0.5766 - val_loss: 0.6639 - val_accuracy: 0.6004
Epoch 7/400
59/59 [==============================] - 0s 4ms/step - loss: 0.6693 - accuracy: 0.5889 - val_loss: 0.6617 - val_accuracy: 0.6154
Epoch 8/400
5

## binary

In [ ]:
X_train = bin_train.drop("Target", axis=1)
y_train = bin_train['Target']

X_test = bin_test.drop("Target", axis=1)
y_test = bin_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [01:19<00:02,  2.90s/it]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027383 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4650
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 1550
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [01:21<00:00,  2.80s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.92               0.92     0.92      0.92   
LGBMClassifier                     0.92               0.92     0.92      0.92   
XGBClassifier                      0.92               0.92     0.92      0.92   
RandomForestClassifier             0.91               0.91     0.91      0.91   
NuSVC                              0.90               0.91     0.91      0.90   
SVC                                0.90               0.90     0.90      0.90   
BernoulliNB                        0.90               0.90     0.90      0.90   
NearestCentroid                    0.89               0.89     0.89      0.89   
BaggingClassifier                  0.89               0.89     0.89      0.89   
Perceptron                         0.86               0.86     0.86      0.86   
SGDClassifier               

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 4s 38ms/step - loss: 0.9365 - accuracy: 0.4936 - val_loss: 0.6736 - val_accuracy: 0.6346
Epoch 2/400
59/59 [==============================] - 1s 25ms/step - loss: 0.7803 - accuracy: 0.5482 - val_loss: 0.6708 - val_accuracy: 0.6709
Epoch 3/400
59/59 [==============================] - 1s 24ms/step - loss: 0.6995 - accuracy: 0.5969 - val_loss: 0.5751 - val_accuracy: 0.8291
Epoch 4/400
59/59 [==============================] - 1s 22ms/step - loss: 0.5379 - accuracy: 0.7398 - val_loss: 0.3831 - val_accuracy: 0.8739
Epoch 5/400
59/59 [==============================] - 1s 17ms/step - loss: 0.3254 - accuracy: 0.8667 - val_loss: 0.2821 - val_accuracy: 0.9038
Epoch 6/400
59/59 [==============================] - 1s 13ms/step - loss: 0.2267 - accuracy: 0.9160 - val_loss: 0.2807 - val_accuracy: 0.8953
Epoch 7/400
59/59 [==============================] - 1s 14ms/step - loss: 0.1402 - accuracy: 0.9529 - val_loss: 0.3077 - val_accuracy: 0.9060
Epoch 

## ngrams

In [ ]:
X_train = ng_train.drop("Target", axis=1)
y_train = ng_train['Target']

X_test = ng_test.drop("Target", axis=1)
y_test = ng_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [00:16<00:00,  1.69it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013903 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 21889
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 397
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [00:17<00:00,  1.62it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
SVC                                0.93               0.93     0.93      0.93   
ExtraTreesClassifier               0.92               0.92     0.92      0.92   
NuSVC                              0.91               0.91     0.91      0.91   
XGBClassifier                      0.91               0.91     0.91      0.91   
RandomForestClassifier             0.91               0.91     0.91      0.91   
LGBMClassifier                     0.90               0.90     0.90      0.90   
RidgeClassifierCV                  0.90               0.90     0.90      0.90   
RidgeClassifier                    0.90               0.90     0.90      0.90   
LinearDiscriminantAnalysis         0.90               0.90     0.90      0.90   
LogisticRegression                 0.89               0.89     0.89      0.89   
QuadraticDiscriminantAnalysi

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 10ms/step - loss: 0.7832 - accuracy: 0.5551 - val_loss: 0.6219 - val_accuracy: 0.8098
Epoch 2/400
59/59 [==============================] - 0s 6ms/step - loss: 0.6360 - accuracy: 0.6681 - val_loss: 0.4000 - val_accuracy: 0.8739
Epoch 3/400
59/59 [==============================] - 0s 7ms/step - loss: 0.4189 - accuracy: 0.8249 - val_loss: 0.2851 - val_accuracy: 0.8868
Epoch 4/400
59/59 [==============================] - 0s 7ms/step - loss: 0.3021 - accuracy: 0.8881 - val_loss: 0.2750 - val_accuracy: 0.8953
Epoch 5/400
59/59 [==============================] - 0s 7ms/step - loss: 0.2404 - accuracy: 0.9160 - val_loss: 0.2758 - val_accuracy: 0.8953
Epoch 6/400
59/59 [==============================] - 0s 7ms/step - loss: 0.1782 - accuracy: 0.9427 - val_loss: 0.3105 - val_accuracy: 0.9017
Epoch 7/400
59/59 [==============================] - 0s 7ms/step - loss: 0.1526 - accuracy: 0.9491 - val_loss: 0.3743 - val_accuracy: 0.9038
Epoch 8/400


## entropy

In [ ]:
X_train = ent_train.drop("Target", axis=1)
y_train = ent_train['Target']

X_test = ent_test.drop("Target", axis=1)
y_test = ent_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

100%|██████████| 29/29 [00:03<00:00,  8.05it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000082 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 1
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
SVC                                0.55               0.54     0.54      0.53   
CalibratedClassifierCV             0.54               0.54     0.54      0.54   
KNeighborsClassifier               0.54               0.54     0.54      0.54   
BaggingClassifier                  0.54               0.54     0.54      0.54   
LabelSpreading                     0.54               0.53     0.53      0.51   

### proposed model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")


Epoch 1/400
59/59 [==============================] - 2s 9ms/step - loss: 0.6958 - accuracy: 0.5037 - val_loss: 0.6954 - val_accuracy: 0.5064
Epoch 2/400
59/59 [==============================] - 0s 4ms/step - loss: 0.6968 - accuracy: 0.4973 - val_loss: 0.6920 - val_accuracy: 0.5214
Epoch 3/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6986 - accuracy: 0.4984 - val_loss: 0.6931 - val_accuracy: 0.5192
Epoch 4/400
59/59 [==============================] - 0s 6ms/step - loss: 0.6957 - accuracy: 0.5016 - val_loss: 0.6932 - val_accuracy: 0.5043
Epoch 5/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6931 - accuracy: 0.5203 - val_loss: 0.6953 - val_accuracy: 0.5021
Epoch 6/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6974 - accuracy: 0.5086 - val_loss: 0.6927 - val_accuracy: 0.5085
Epoch 7/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6946 - accuracy: 0.5075 - val_loss: 0.6929 - val_accuracy: 0.5150
Epoch 8/400
5

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

F1 Score: 0.6683


In [ ]:
import pandas as pd

# Assuming your dataset is a DataFrame named 'df1' with columns as feature names
# Example feature names: 'binary1', 'binary2', ..., 'QSO2_G2', 'Target'

# Create a dictionary to store separated DataFrames
feature_groups = {}

# Iterate through columns (features) in the DataFrame
for column in df1.columns:
    # Extract the character name by splitting the feature name
    character_name = ''.join(filter(str.isalpha, column.split('_')[0]))

    # Check if the character name is already a key in the dictionary
    if character_name in feature_groups:
        # If yes, append the column to the existing DataFrame
        feature_groups[character_name][column] = df1[column]
    else:
        # If no, create a new DataFrame for that character name
        feature_groups[character_name] = df1[[column]]

# Now, feature_groups contains separated DataFrames based on character names
# Access each DataFrame using feature_groups['CHARACTER_NAME']

# Example usage:
# Access the DataFrame for features with character name 'binary'
f = feature_groups['aaindex']

### separate dataset

In [ ]:
aac_test = df2[list(f)+ ['Target']]

In [ ]:
aac_train = df1[list(f)+ ['Target']]

In [ ]:
qso_train = df1[list(f)+ ['Target']]


In [ ]:
qso_test = df2[list(f)+ ['Target']]

In [ ]:
aa1_train = df1[list(f)+ ['Target']]

In [ ]:
aa1_test = df2[list(f)+ ['Target']]

In [ ]:
ctdc_train = df1[list(f)+ ['Target']]

In [ ]:
ctdc_test = df2[list(f)+ ['Target']]

In [ ]:
ent_train = df1[list(f)+ ['Target']]

In [ ]:
ent_test = df2[list(f)+ ['Target']]

In [ ]:
mr_train = df1[list(f)+ ['Target']]

In [ ]:
mr_test = df2[list(f)+ ['Target']]

In [ ]:
paac_train = df1[list(f)+ ['Target']]

In [ ]:
paac_test = df2[list(f)+ ['Target']]

In [ ]:
ng_train = df1[list(f)+ ['Target']]

In [ ]:
ng_test = df2[list(f)+ ['Target']]

In [ ]:
bin_train = df1[list(f)+ ['Target']]

In [ ]:
bin_test = df2[list(f)+ ['Target']]

## lazzyporedict

## aac individual

In [ ]:
X_train = aac_train.drop("Target", axis=1)
y_train = aac_train['Target']

X_test = aac_test.drop("Target", axis=1)
y_test = aac_test['Target']

### lazzy clasify

In [ ]:
!python -m pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
pip install lazypredict

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [00:12<00:00,  1.31it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000202 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4784
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [00:13<00:00,  2.21it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.88               0.88     0.88      0.88   
LGBMClassifier                     0.88               0.88     0.88      0.88   
RandomForestClassifier             0.88               0.88     0.88      0.88   
XGBClassifier                      0.87               0.87     0.87      0.87   
BaggingClassifier                  0.85               0.85     0.85      0.85   
SVC                                0.85               0.85     0.85      0.85   
NuSVC                              0.84               0.84     0.84      0.84   
LabelPropagation                   0.83               0.83     0.83      0.83   
LabelSpreading                     0.83               0.83     0.83      0.83   
KNeighborsClassifier               0.83               0.83     0.83      0.83   
QuadraticDiscriminantAnalysi

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 9ms/step - loss: 0.6861 - accuracy: 0.5707 - val_loss: 0.5916 - val_accuracy: 0.7500
Epoch 2/400
59/59 [==============================] - 0s 6ms/step - loss: 0.6120 - accuracy: 0.6879 - val_loss: 0.4985 - val_accuracy: 0.7543
Epoch 3/400
59/59 [==============================] - 0s 5ms/step - loss: 0.5427 - accuracy: 0.7537 - val_loss: 0.4887 - val_accuracy: 0.7970
Epoch 4/400
59/59 [==============================] - 0s 6ms/step - loss: 0.5340 - accuracy: 0.7645 - val_loss: 0.4636 - val_accuracy: 0.8013
Epoch 5/400
59/59 [==============================] - 0s 6ms/step - loss: 0.5084 - accuracy: 0.7827 - val_loss: 0.4469 - val_accuracy: 0.7991
Epoch 6/400
59/59 [==============================] - 0s 6ms/step - loss: 0.4903 - accuracy: 0.8003 - val_loss: 0.4548 - val_accuracy: 0.8120
Epoch 7/400
59/59 [==============================] - 0s 6ms/step - loss: 0.4836 - accuracy: 0.8019 - val_loss: 0.4263 - val_accuracy: 0.8098
Epoch 8/400
5

## QSO

In [ ]:
X_train = qso_train.drop("Target", axis=1)
y_train = qso_train['Target']

X_test = qso_test.drop("Target", axis=1)
y_test = qso_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [00:08<00:00,  3.99it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000913 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6266
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 44
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [00:08<00:00,  3.30it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.90               0.90     0.90      0.90   
RandomForestClassifier             0.89               0.89     0.89      0.89   
LGBMClassifier                     0.89               0.89     0.89      0.89   
XGBClassifier                      0.88               0.88     0.88      0.88   
SVC                                0.86               0.86     0.86      0.86   
NuSVC                              0.85               0.85     0.85      0.85   
BaggingClassifier                  0.85               0.85     0.85      0.85   
LabelPropagation                   0.84               0.84     0.84      0.84   
LabelSpreading                     0.84               0.84     0.84      0.84   
KNeighborsClassifier               0.84               0.84     0.84      0.84   
RidgeClassifierCV           

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 8ms/step - loss: 0.7029 - accuracy: 0.5466 - val_loss: 0.5964 - val_accuracy: 0.7543
Epoch 2/400
59/59 [==============================] - 0s 6ms/step - loss: 0.5640 - accuracy: 0.7211 - val_loss: 0.4964 - val_accuracy: 0.7735
Epoch 3/400
59/59 [==============================] - 0s 5ms/step - loss: 0.5179 - accuracy: 0.7768 - val_loss: 0.4477 - val_accuracy: 0.7906
Epoch 4/400
59/59 [==============================] - 0s 6ms/step - loss: 0.4887 - accuracy: 0.7928 - val_loss: 0.4293 - val_accuracy: 0.7970
Epoch 5/400
59/59 [==============================] - 0s 5ms/step - loss: 0.4532 - accuracy: 0.8014 - val_loss: 0.4208 - val_accuracy: 0.8034
Epoch 6/400
59/59 [==============================] - 0s 5ms/step - loss: 0.4316 - accuracy: 0.8094 - val_loss: 0.4071 - val_accuracy: 0.8077
Epoch 7/400
59/59 [==============================] - 0s 6ms/step - loss: 0.4000 - accuracy: 0.8217 - val_loss: 0.4021 - val_accuracy: 0.8269
Epoch 8/400
5

## paac

In [ ]:
X_train = paac_train.drop("Target", axis=1)
y_train = paac_train['Target']

X_test = paac_test.drop("Target", axis=1)
y_test = paac_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 93%|█████████▎| 27/29 [00:05<00:00,  6.77it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000251 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5294
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [00:05<00:00,  4.94it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
XGBClassifier                      0.88               0.88     0.88      0.88   
ExtraTreesClassifier               0.88               0.88     0.88      0.88   
RandomForestClassifier             0.88               0.88     0.88      0.88   
LGBMClassifier                     0.88               0.88     0.88      0.88   
BaggingClassifier                  0.87               0.87     0.87      0.87   
SVC                                0.85               0.85     0.85      0.85   
NuSVC                              0.85               0.85     0.85      0.85   
LabelSpreading                     0.83               0.83     0.83      0.83   
LabelPropagation                   0.83               0.83     0.83      0.83   
KNeighborsClassifier               0.81               0.81     0.81      0.81   
QuadraticDiscriminantAnalysi

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 8ms/step - loss: 0.6922 - accuracy: 0.5653 - val_loss: 0.6004 - val_accuracy: 0.7628
Epoch 2/400
59/59 [==============================] - 0s 6ms/step - loss: 0.5938 - accuracy: 0.6874 - val_loss: 0.5123 - val_accuracy: 0.7671
Epoch 3/400
59/59 [==============================] - 0s 6ms/step - loss: 0.5487 - accuracy: 0.7650 - val_loss: 0.5037 - val_accuracy: 0.7842
Epoch 4/400
59/59 [==============================] - 0s 6ms/step - loss: 0.5155 - accuracy: 0.7746 - val_loss: 0.4746 - val_accuracy: 0.7863
Epoch 5/400
59/59 [==============================] - 1s 9ms/step - loss: 0.5002 - accuracy: 0.7966 - val_loss: 0.4575 - val_accuracy: 0.7927
Epoch 6/400
59/59 [==============================] - 1s 9ms/step - loss: 0.4665 - accuracy: 0.8014 - val_loss: 0.4466 - val_accuracy: 0.7991
Epoch 7/400
59/59 [==============================] - 1s 10ms/step - loss: 0.4790 - accuracy: 0.7955 - val_loss: 0.4555 - val_accuracy: 0.7885
Epoch 8/400


## aaindex

In [ ]:
X_train = aa1_train.drop("Target", axis=1)
y_train = aa1_train['Target']

X_test = aa1_test.drop("Target", axis=1)
y_test = aa1_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [01:30<00:03,  3.05s/it]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017934 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 141012
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 553
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [01:40<00:00,  3.48s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
QuadraticDiscriminantAnalysis      1.00               1.00     1.00      1.00   
XGBClassifier                      0.92               0.92     0.92      0.92   
LGBMClassifier                     0.90               0.90     0.90      0.90   
NuSVC                              0.89               0.89     0.89      0.89   
SVC                                0.88               0.88     0.88      0.88   
ExtraTreesClassifier               0.88               0.88     0.88      0.88   
RandomForestClassifier             0.87               0.87     0.87      0.87   
KNeighborsClassifier               0.84               0.84     0.84      0.84   
BaggingClassifier                  0.83               0.83     0.83      0.83   
DecisionTreeClassifier             0.79               0.79     0.79      0.79   
AdaBoostClassifier          

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 10ms/step - loss: 0.9208 - accuracy: 0.5171 - val_loss: 0.6766 - val_accuracy: 0.5876
Epoch 2/400
59/59 [==============================] - 0s 8ms/step - loss: 0.8023 - accuracy: 0.5262 - val_loss: 0.6617 - val_accuracy: 0.6026
Epoch 3/400
59/59 [==============================] - 1s 9ms/step - loss: 0.7773 - accuracy: 0.5230 - val_loss: 0.6568 - val_accuracy: 0.6838
Epoch 4/400
59/59 [==============================] - 1s 9ms/step - loss: 0.7353 - accuracy: 0.5616 - val_loss: 0.6444 - val_accuracy: 0.7073
Epoch 5/400
59/59 [==============================] - 1s 8ms/step - loss: 0.7031 - accuracy: 0.5723 - val_loss: 0.6370 - val_accuracy: 0.7158
Epoch 6/400
59/59 [==============================] - 0s 8ms/step - loss: 0.6880 - accuracy: 0.6146 - val_loss: 0.6165 - val_accuracy: 0.7051
Epoch 7/400
59/59 [==============================] - 1s 9ms/step - loss: 0.6737 - accuracy: 0.6156 - val_loss: 0.5977 - val_accuracy: 0.7393
Epoch 8/400


## CTDC

In [ ]:
X_train = ctdc_train.drop("Target", axis=1)
y_train = ctdc_train['Target']

X_test = ctdc_test.drop("Target", axis=1)
y_test = ctdc_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [00:08<00:00,  3.58it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001448 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9929
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [00:09<00:00,  3.04it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.87               0.86     0.86      0.87   
RandomForestClassifier             0.85               0.85     0.85      0.85   
LGBMClassifier                     0.85               0.85     0.85      0.85   
KNeighborsClassifier               0.84               0.84     0.84      0.84   
XGBClassifier                      0.84               0.84     0.84      0.84   
SVC                                0.83               0.83     0.83      0.83   
NuSVC                              0.83               0.83     0.83      0.83   
LabelPropagation                   0.83               0.83     0.83      0.83   
LabelSpreading                     0.83               0.83     0.83      0.83   
BaggingClassifier                  0.82               0.82     0.82      0.82   
AdaBoostClassifier          

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 8ms/step - loss: 0.6598 - accuracy: 0.6322 - val_loss: 0.5419 - val_accuracy: 0.7842
Epoch 2/400
59/59 [==============================] - 0s 5ms/step - loss: 0.5622 - accuracy: 0.7468 - val_loss: 0.5101 - val_accuracy: 0.7906
Epoch 3/400
59/59 [==============================] - 0s 6ms/step - loss: 0.5376 - accuracy: 0.7687 - val_loss: 0.4994 - val_accuracy: 0.7949
Epoch 4/400
59/59 [==============================] - 0s 6ms/step - loss: 0.5291 - accuracy: 0.7693 - val_loss: 0.5088 - val_accuracy: 0.7906
Epoch 5/400
59/59 [==============================] - 0s 6ms/step - loss: 0.5070 - accuracy: 0.7827 - val_loss: 0.4897 - val_accuracy: 0.7927
Epoch 6/400
59/59 [==============================] - 0s 7ms/step - loss: 0.5048 - accuracy: 0.7923 - val_loss: 0.4830 - val_accuracy: 0.7927
Epoch 7/400
59/59 [==============================] - 0s 6ms/step - loss: 0.4990 - accuracy: 0.7843 - val_loss: 0.4875 - val_accuracy: 0.7885
Epoch 8/400
5

## moran

In [ ]:
X_train = mr_train.drop("Target", axis=1)
y_train = mr_train['Target']

X_test = mr_test.drop("Target", axis=1)
y_test = mr_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [00:06<00:00,  2.88it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000368 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [00:07<00:00,  4.11it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.62               0.62     0.62      0.62   
NuSVC                              0.61               0.61     0.61      0.61   
SVC                                0.61               0.61     0.61      0.60   
LabelSpreading                     0.60               0.60     0.60      0.60   
LabelPropagation                   0.60               0.60     0.60      0.60   
RandomForestClassifier             0.60               0.60     0.60      0.60   
LGBMClassifier                     0.60               0.60     0.60      0.60   
BaggingClassifier                  0.58               0.58     0.58      0.58   
ExtraTreeClassifier                0.58               0.58     0.58      0.58   
XGBClassifier                      0.58               0.58     0.58      0.58   
KNeighborsClassifier        

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 9ms/step - loss: 0.7037 - accuracy: 0.4855 - val_loss: 0.6850 - val_accuracy: 0.5662
Epoch 2/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6951 - accuracy: 0.5278 - val_loss: 0.6767 - val_accuracy: 0.5620
Epoch 3/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6897 - accuracy: 0.5177 - val_loss: 0.6727 - val_accuracy: 0.5940
Epoch 4/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6894 - accuracy: 0.5380 - val_loss: 0.6716 - val_accuracy: 0.5769
Epoch 5/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6877 - accuracy: 0.5252 - val_loss: 0.6714 - val_accuracy: 0.5598
Epoch 6/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6781 - accuracy: 0.5482 - val_loss: 0.6661 - val_accuracy: 0.5812
Epoch 7/400
59/59 [==============================] - 0s 6ms/step - loss: 0.6794 - accuracy: 0.5412 - val_loss: 0.6645 - val_accuracy: 0.6175
Epoch 8/400
5

## binary

In [ ]:
X_train = bin_train.drop("Target", axis=1)
y_train = bin_train['Target']

X_test = bin_test.drop("Target", axis=1)
y_test = bin_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [01:57<00:03,  3.48s/it]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.041438 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4692
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 1564
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [01:59<00:00,  4.11s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.89               0.89     0.89      0.89   
RandomForestClassifier             0.87               0.87     0.87      0.87   
LGBMClassifier                     0.87               0.87     0.87      0.87   
NuSVC                              0.85               0.85     0.85      0.85   
XGBClassifier                      0.85               0.85     0.85      0.85   
SVC                                0.85               0.85     0.85      0.85   
NearestCentroid                    0.81               0.81     0.81      0.81   
Perceptron                         0.81               0.81     0.81      0.81   
BernoulliNB                        0.81               0.81     0.81      0.81   
BaggingClassifier                  0.80               0.80     0.80      0.80   
SGDClassifier               

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 3s 17ms/step - loss: 0.9047 - accuracy: 0.5155 - val_loss: 0.6771 - val_accuracy: 0.6132
Epoch 2/400
59/59 [==============================] - 1s 15ms/step - loss: 0.8096 - accuracy: 0.5278 - val_loss: 0.6737 - val_accuracy: 0.5897
Epoch 3/400
59/59 [==============================] - 1s 13ms/step - loss: 0.7313 - accuracy: 0.5535 - val_loss: 0.6502 - val_accuracy: 0.7415
Epoch 4/400
59/59 [==============================] - 1s 16ms/step - loss: 0.6576 - accuracy: 0.6461 - val_loss: 0.5841 - val_accuracy: 0.7650
Epoch 5/400
59/59 [==============================] - 1s 16ms/step - loss: 0.5352 - accuracy: 0.7264 - val_loss: 0.4693 - val_accuracy: 0.7842
Epoch 6/400
59/59 [==============================] - 1s 16ms/step - loss: 0.3823 - accuracy: 0.8324 - val_loss: 0.4166 - val_accuracy: 0.7756
Epoch 7/400
59/59 [==============================] - 1s 16ms/step - loss: 0.2889 - accuracy: 0.8822 - val_loss: 0.4249 - val_accuracy: 0.8077
Epoch 

## ngrams

In [ ]:
X_train = ng_train.drop("Target", axis=1)
y_train = ng_train['Target']

X_test = ng_test.drop("Target", axis=1)
y_test = ng_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

 97%|█████████▋| 28/29 [00:22<00:00,  1.41it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011528 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 21694
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 398
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 29/29 [00:23<00:00,  1.24it/s]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.89               0.89     0.89      0.89   
SVC                                0.88               0.88     0.88      0.88   
NuSVC                              0.88               0.88     0.88      0.88   
RandomForestClassifier             0.87               0.87     0.87      0.87   
XGBClassifier                      0.87               0.87     0.87      0.87   
LGBMClassifier                     0.87               0.87     0.87      0.87   
QuadraticDiscriminantAnalysis      0.83               0.83     0.83      0.82   
RidgeClassifierCV                  0.83               0.83     0.83      0.83   
BaggingClassifier                  0.82               0.82     0.82      0.82   
RidgeClassifier                    0.82               0.82     0.82      0.82   
LinearDiscriminantAnalysis  

### MLAFP-XN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")

Epoch 1/400
59/59 [==============================] - 2s 12ms/step - loss: 0.8529 - accuracy: 0.5219 - val_loss: 0.6793 - val_accuracy: 0.5235
Epoch 2/400
59/59 [==============================] - 1s 9ms/step - loss: 0.6930 - accuracy: 0.5701 - val_loss: 0.6187 - val_accuracy: 0.7521
Epoch 3/400
59/59 [==============================] - 1s 12ms/step - loss: 0.6107 - accuracy: 0.6660 - val_loss: 0.5007 - val_accuracy: 0.8013
Epoch 4/400
59/59 [==============================] - 1s 13ms/step - loss: 0.5102 - accuracy: 0.7677 - val_loss: 0.4331 - val_accuracy: 0.8248
Epoch 5/400
59/59 [==============================] - 1s 13ms/step - loss: 0.4321 - accuracy: 0.8062 - val_loss: 0.4246 - val_accuracy: 0.8226
Epoch 6/400
59/59 [==============================] - 1s 12ms/step - loss: 0.3601 - accuracy: 0.8517 - val_loss: 0.4101 - val_accuracy: 0.8419
Epoch 7/400
59/59 [==============================] - 1s 11ms/step - loss: 0.2935 - accuracy: 0.8951 - val_loss: 0.4332 - val_accuracy: 0.8333
Epoch 8

## entropy

In [ ]:
X_train = ent_train.drop("Target", axis=1)
y_train = ent_train['Target']

X_test = ent_test.drop("Target", axis=1)
y_test = ent_test['Target']

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a dataset (X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LazyClassifier
clf = LazyClassifier()

# Fit and predict
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)
results_df = pd.DataFrame(models)
results_df.to_csv('a.csv', index=False)

100%|██████████| 29/29 [00:04<00:00,  6.74it/s]

[LightGBM] [Info] Number of positive: 1168, number of negative: 1168
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000095 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 2336, number of used features: 1
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
SVC                                0.57               0.57     0.57      0.57   
RandomForestClassifier             0.57               0.57     0.57      0.57   
DecisionTreeClassifier             0.56               0.56     0.56      0.56   
ExtraTreesClassifier               0.56               0.56     0.56      0.56   
BaggingClassifier                  0.56               0.56     0.56      0.56   

### proposed model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc
)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score
# Assuming X_train1, y_train1, X_test1, y_test1 are defined

# Standardize the data
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train)
X_test1 = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(400, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=400, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Make predictions on the test set
test_predictions = (model.predict(X_test1) > 0.5).astype(int)

# Evaluate the performance on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Testing Accuracy: {test_accuracy}")



# Calculate sensitivity, specificity, and AUC score
false_positive_rate, true_positive_rate, _ = roc_curve(y_test, model.predict(X_test1))
roc_auc = auc(false_positive_rate, true_positive_rate)


from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

mcc = matthews_corrcoef(y_test, test_predictions)

# Calculate Kappa score
kappa = cohen_kappa_score(y_test, test_predictions)

f1 = f1_score(y_test, test_predictions)

print(f"F1 Score: {f1:.4f}")

print(f"Testing Accuracy: {test_accuracy}")
print(f"\nAUC Score: {roc_auc:.4f}")

print(f"MCC Score: {mcc:.4f}")
print(f"Kappa Score: {kappa:.4f}")


Epoch 1/400
59/59 [==============================] - 2s 10ms/step - loss: 0.6961 - accuracy: 0.4893 - val_loss: 0.6901 - val_accuracy: 0.5171
Epoch 2/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6977 - accuracy: 0.4968 - val_loss: 0.6894 - val_accuracy: 0.5043
Epoch 3/400
59/59 [==============================] - 0s 6ms/step - loss: 0.6963 - accuracy: 0.5059 - val_loss: 0.6898 - val_accuracy: 0.5470
Epoch 4/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6937 - accuracy: 0.5236 - val_loss: 0.6902 - val_accuracy: 0.5363
Epoch 5/400
59/59 [==============================] - 0s 6ms/step - loss: 0.6918 - accuracy: 0.5187 - val_loss: 0.6888 - val_accuracy: 0.5299
Epoch 6/400
59/59 [==============================] - 0s 5ms/step - loss: 0.6930 - accuracy: 0.5316 - val_loss: 0.6891 - val_accuracy: 0.5235
Epoch 7/400
59/59 [==============================] - 0s 6ms/step - loss: 0.6933 - accuracy: 0.5037 - val_loss: 0.6896 - val_accuracy: 0.5342
Epoch 8/400
